<a href="https://colab.research.google.com/github/hamdansyaif/handson-ml2/blob/main/midterm-ml/transaction/midterm_Transaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import polars as pl

BASE_PATH = "/content/drive/MyDrive/midterm_Tansaction"

train_transaction = pl.read_csv(f"{BASE_PATH}/train_transaction.csv")
test_transaction  = pl.read_csv(f"{BASE_PATH}/test_transaction.csv")

print(train_transaction.shape) #590540 row dan 393 feature + 1 target
print(test_transaction.shape) #506691 row dan 393 feature

(590540, 394)
(506691, 393)


In [2]:
train_transaction.head() #isFraud adalah target featurenya

TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,…,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
i64,i64,i64,f64,str,i64,f64,f64,str,f64,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2987000,0,86400,68.5,"""W""",13926,null,150.0,"""discover""",142.0,"""credit""",315.0,87.0,19.0,null,null,null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,null,13.0,null,null,null,…,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987001,0,86401,29.0,"""W""",2755,404.0,150.0,"""mastercard""",102.0,"""credit""",325.0,87.0,null,null,"""gmail.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,null,null,0.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987002,0,86469,59.0,"""W""",4663,490.0,150.0,"""visa""",166.0,"""debit""",330.0,87.0,287.0,null,"""outlook.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,null,null,0.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987003,0,86499,50.0,"""W""",18132,567.0,150.0,"""mastercard""",117.0,"""debit""",476.0,87.0,null,null,"""yahoo.com""",null,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,null,…,0.0,0.0,1.0,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987004,0,86506,50.0,"""H""",4497,514.0,150.0,"""mastercard""",102.0,"""credit""",420.0,87.0,null,null,"""gmail.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,null,null,null,null,null,…,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
test_transaction.head() #uji data baru, setelah pembuatan model machine learningnya

TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,…,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
i64,i64,f64,str,i64,f64,f64,str,f64,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3663549,18403224,31.95,"""W""",10409,111.0,150.0,"""visa""",226.0,"""debit""",170.0,87.0,1.0,null,"""gmail.com""",null,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,null,null,…,0.0,0.0,1.0,0.0,47.950001,0.0,0.0,47.950001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663550,18403263,49.0,"""W""",4272,111.0,150.0,"""visa""",226.0,"""debit""",299.0,87.0,4.0,null,"""aol.com""",null,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,null,null,…,0.0,0.0,1.0,0.0,280.0,77.0,0.0,280.0,0.0,77.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663551,18403310,171.0,"""W""",4476,574.0,150.0,"""visa""",226.0,"""debit""",472.0,87.0,2635.0,null,"""hotmail.com""",null,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,null,null,…,0.0,0.0,1.0,0.0,1321.0,0.0,0.0,1058.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663552,18403310,284.95,"""W""",10989,360.0,150.0,"""visa""",166.0,"""debit""",205.0,87.0,17.0,null,"""gmail.com""",null,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,282.540009,282.540009,282.540009,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663553,18403317,67.95,"""W""",18018,452.0,150.0,"""mastercard""",117.0,"""debit""",264.0,87.0,6.0,null,"""gmail.com""",null,6.0,6.0,0.0,0.0,2.0,5.0,0.0,0.0,5.0,0.0,6.0,0.0,14.0,6.0,22.0,22.0,0.0,22.0,0.0,null,null,…,0.0,0.0,1.0,67.949997,183.850006,67.949997,67.949997,183.850006,67.949997,67.949997,67.949997,183.850006,67.949997,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


# EDA

In [4]:
# --- CODE BLOCK 1: EDA Target & Missing Values ---

# 1. Cek Distribusi Target (Seberapa Imbalance?)
fraud_counts = train_transaction["isFraud"].value_counts()
total_rows = train_transaction.height

print("=== DISTRIBUSI TARGET (isFraud) ===")
print(fraud_counts)

# Hitung persentase
fraud_ratio = train_transaction["isFraud"].mean() * 100
print(f"\nPersentase Fraud: {fraud_ratio:.2f}%")
print(f"Persentase Normal: {100 - fraud_ratio:.2f}%")

# 2. Cek Overview Missing Values
# Kita hitung ada berapa kolom yang missing value-nya di atas 50%
null_counts = train_transaction.null_count()
null_ratios = null_counts.row(0) # Mengambil baris pertama sebagai tuple

# Identifikasi kolom dengan banyak missing values
cols_with_many_nulls = [
    train_transaction.columns[i]
    for i, val in enumerate(null_ratios)
    if (val / total_rows) > 0.5
]

print(f"\n=== MISSING VALUES OVERVIEW ===")
print(f"Total Kolom: {len(train_transaction.columns)}")
print(f"Jumlah kolom dengan Missing Value > 50%: {len(cols_with_many_nulls)}")
print("5 Contoh kolom yang kosong melompong (>50% null):", cols_with_many_nulls[:5])

=== DISTRIBUSI TARGET (isFraud) ===
shape: (2, 2)
┌─────────┬────────┐
│ isFraud ┆ count  │
│ ---     ┆ ---    │
│ i64     ┆ u32    │
╞═════════╪════════╡
│ 0       ┆ 569877 │
│ 1       ┆ 20663  │
└─────────┴────────┘

Persentase Fraud: 3.50%
Persentase Normal: 96.50%

=== MISSING VALUES OVERVIEW ===
Total Kolom: 394
Jumlah kolom dengan Missing Value > 50%: 174
5 Contoh kolom yang kosong melompong (>50% null): ['dist1', 'dist2', 'R_emaildomain', 'D5', 'D6']


# Preprocessing

## Feature Engineering

In [5]:
# --- CODE BLOCK 2: Feature Engineering (Time) ---

def make_time_features(df):
    # Asumsi TransactionDT adalah detik.
    # Kita buat fitur: hour (jam dalam hari) dan day (hari ke-sekian)
    df = df.with_columns([
        (pl.col("TransactionDT") // 3600 % 24).alias("Transaction_Hour"),
        (pl.col("TransactionDT") // (3600 * 24) % 7).alias("Transaction_Day")
    ])
    return df

# Terapkan ke Train dan Test
train_transaction = make_time_features(train_transaction)
test_transaction = make_time_features(test_transaction)

print("=== TIME FEATURES CREATED ===")
print("Train Columns Updated:", train_transaction.select(['TransactionDT', 'Transaction_Hour', 'Transaction_Day']).head())

=== TIME FEATURES CREATED ===
Train Columns Updated: shape: (5, 3)
┌───────────────┬──────────────────┬─────────────────┐
│ TransactionDT ┆ Transaction_Hour ┆ Transaction_Day │
│ ---           ┆ ---              ┆ ---             │
│ i64           ┆ i64              ┆ i64             │
╞═══════════════╪══════════════════╪═════════════════╡
│ 86400         ┆ 0                ┆ 1               │
│ 86401         ┆ 0                ┆ 1               │
│ 86469         ┆ 0                ┆ 1               │
│ 86499         ┆ 0                ┆ 1               │
│ 86506         ┆ 0                ┆ 1               │
└───────────────┴──────────────────┴─────────────────┘


## Encoding & Filling NaNs

In [6]:
# --- CODE BLOCK 3 (FIXED): Preprocessing & Saving to Drive ---

print("Sedang melakukan preprocessing ulang... mohon tunggu sebentar.")

# 1. Identifikasi Kolom Kategorikal
cat_cols = [col for col, dtype in train_transaction.schema.items() if dtype == pl.String]

# 2. Encoding & Filling Nulls
with pl.StringCache():
    # --- PROSES TRAIN DATA ---
    train_final = train_transaction.with_columns([
        pl.col(c).cast(pl.Categorical).to_physical().alias(c)
        for c in cat_cols
    ]).fill_null(-1)

    # --- PROSES TEST DATA ---
    test_final = test_transaction.with_columns([
        pl.col(c).cast(pl.Categorical).to_physical().alias(c)
        for c in cat_cols
    ]).fill_null(-1)

print("Preprocessing selesai!")

# Cek Total Nulls (Versi Polars yang Benar)
# Kita ambil baris pertama (row 0) dari null_count dan jumlahkan manual
total_nulls = sum(train_final.null_count().row(0))
print(f"Total Missing Values di Train Final: {total_nulls}")
# Seharusnya 0 karena sudah di-fill(-1)

# 3. SIMPAN KE GOOGLE DRIVE
print("\nSedang menyimpan file clean ke Google Drive...")

try:
    # Simpan Train
    train_final.write_csv(f"{BASE_PATH}/train_clean.csv")
    print(f"Sukses! File tersimpan di: {BASE_PATH}/train_clean.csv")

    # Simpan Test
    test_final.write_csv(f"{BASE_PATH}/test_clean.csv")
    print(f"Sukses! File tersimpan di: {BASE_PATH}/test_clean.csv")

except Exception as e:
    print(f"Gagal menyimpan: {e}")
    print("Pastikan Google Drive sudah dimount dengan benar.")

print("\n=== DATA BERSIH SUDAH AMAN, LANJUT STEP BERIKUTNYA ===")

Sedang melakukan preprocessing ulang... mohon tunggu sebentar.
Preprocessing selesai!
Total Missing Values di Train Final: 0

Sedang menyimpan file clean ke Google Drive...
Sukses! File tersimpan di: /content/drive/MyDrive/midterm_Tansaction/train_clean.csv
Sukses! File tersimpan di: /content/drive/MyDrive/midterm_Tansaction/test_clean.csv

=== DATA BERSIH SUDAH AMAN, LANJUT STEP BERIKUTNYA ===


# Data Splitting

In [7]:
# --- CODE BLOCK 4 (UPDATED): Load Clean Data & Time-Series Split ---

print("1. Sedang me-load data bersih dari Drive...")
# Kita load file yang sudah bersih (clean) agar aman jika Colab restart
train_final = pl.read_csv(f"{BASE_PATH}/train_clean.csv")
test_final = pl.read_csv(f"{BASE_PATH}/test_clean.csv")

print(f"Data Loaded: {train_final.shape}")

print("\n2. Sedang membagi data (Splitting Time-Series)...")

# A. Pastikan data urut berdasarkan waktu (TransactionDT)
# Kritis untuk mencegah "Future Leakage" (Model nyontek masa depan)
train_final = train_final.sort("TransactionDT")

# B. Hitung titik potong (80% Train, 20% Validation)
total_rows = train_final.height
split_index = int(total_rows * 0.8)

# C. Lakukan Slicing (Pemotongan)
# Train: 80% data awal (Masa Lalu)
X_train_full = train_final[:split_index]
# Val: 20% data akhir (Masa Depan - Simulasi Ujian)
X_val_full = train_final[split_index:]

# D. Pisahkan Fitur (X) dan Target (y)
y_train = X_train_full["isFraud"]
y_val = X_val_full["isFraud"]

# Hapus kolom curang/tidak perlu:
# - isFraud: Kunci jawaban (harus dipisah)
# - TransactionID: Cuma nomor urut (noise)
# - TransactionDT: Sudah diwakili oleh Transaction_Hour & Transaction_Day
drop_cols = ["isFraud", "TransactionID", "TransactionDT"]

X_train = X_train_full.drop(drop_cols)
X_val = X_val_full.drop(drop_cols)

# Siapkan juga data Test untuk Submission nanti (hapus ID & DT saja)
X_test_submission = test_final.drop(["TransactionID", "TransactionDT"])

print("=== SPLITTING & PREPARATION SELESAI ===")
print(f"Total Data Train (Belajar) : {X_train.height} baris")
print(f"Total Data Val   (Ujian)   : {X_val.height} baris")
print(f"Jumlah Fitur Masuk Model   : {len(X_train.columns)} kolom")

# Verifikasi Memory
import gc
del train_final, X_train_full, X_val_full
gc.collect() # Bersihkan RAM karena variabel aslinya sudah tidak dipakai
print("RAM sudah dibersihkan untuk efisiensi.")

1. Sedang me-load data bersih dari Drive...
Data Loaded: (590540, 396)

2. Sedang membagi data (Splitting Time-Series)...
=== SPLITTING & PREPARATION SELESAI ===
Total Data Train (Belajar) : 472432 baris
Total Data Val   (Ujian)   : 118108 baris
Jumlah Fitur Masuk Model   : 393 kolom
RAM sudah dibersihkan untuk efisiensi.


# Modeling

## LightGBM

In [8]:
# --- CODE BLOCK 5: Training Machine Learning (LightGBM) ---

import lightgbm as lgb
from sklearn.metrics import roc_auc_score

print("Sedang menyiapkan data untuk LightGBM...")

# Konversi ke format Dataset LightGBM
# Kita convert ke Pandas sebentar karena LightGBM paling stabil dengan Pandas/Numpy
train_data = lgb.Dataset(X_train.to_pandas(), label=y_train.to_pandas())
val_data = lgb.Dataset(X_val.to_pandas(), label=y_val.to_pandas(), reference=train_data)

# Parameter Model (Resep Masak)
params = {
    'objective': 'binary',           # Klasifikasi 2 kelas (Fraud / Normal)
    'metric': 'auc',                 # Fokus ke AUC Score, bukan Accuracy
    'is_unbalance': True,            # PENTING! Karena fraud cuma 3.5%
    'boosting_type': 'gbdt',         # Algoritma standar (Gradient Boosting)
    'learning_rate': 0.05,           # Kecepatan belajar (jangan terlalu cepat biar teliti)
    'num_leaves': 64,                # Kompleksitas pohon keputusan
    'feature_fraction': 0.9,         # Gunakan 90% fitur tiap ronde (mencegah overfit)
    'bagging_fraction': 0.8,         # Gunakan 80% data tiap ronde
    'bagging_freq': 5,
    'verbose': -1                    # Supaya log tidak berisik
}

print("Mulai Training LightGBM... (Biasanya 1-3 menit)")

# Proses Training
# early_stopping_rounds=50: Kalau score gak naik-naik selama 50 ronde, stop aja (hemat waktu)
model_lgb = lgb.train(
    params,
    train_data,
    num_boost_round=1000,            # Maksimal 1000 ronde
    valid_sets=[train_data, val_data],
    valid_names=['train', 'valid'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=50) # Lapor progress tiap 50 ronde
    ]
)

print("\n=== TRAINING SELESAI ===")
print(f"Best Iteration: {model_lgb.best_iteration}")
print(f"Best AUC Score (Validation): {model_lgb.best_score['valid']['auc']:.4f}")

Sedang menyiapkan data untuk LightGBM...
Mulai Training LightGBM... (Biasanya 1-3 menit)
Training until validation scores don't improve for 50 rounds
[50]	train's auc: 0.936356	valid's auc: 0.897903
[100]	train's auc: 0.953187	valid's auc: 0.909604
[150]	train's auc: 0.963509	valid's auc: 0.912303
[200]	train's auc: 0.970643	valid's auc: 0.913261
Early stopping, best iteration is:
[190]	train's auc: 0.969466	valid's auc: 0.913461

=== TRAINING SELESAI ===
Best Iteration: 190
Best AUC Score (Validation): 0.9135


# Deep Learning Prep

In [9]:
# --- CODE BLOCK HEMAT RAM: Deep Learning (End-to-End) ---

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import numpy as np
import gc # Tukang bersih-bersih RAM

# Pastikan Runtime sudah di-Restart sebelum run ini!
print("=== MULAI DEEP LEARNING MODE HEMAT RAM ===")

# 1. SETUP DATA (Langsung Numpy, Skip Pandas)
print("1. Mengonversi ke Numpy (Hemat Memori)...")
# Kita ambil data yg sudah ada di step splitting tadi
# Gunakan .to_numpy() bukan .to_pandas() karena jauh lebih ringan
X_train_np = X_train.to_numpy().astype(np.float32)
y_train_np = y_train.to_numpy().astype(np.float32)
X_val_np = X_val.to_numpy().astype(np.float32)
y_val_np = y_val.to_numpy().astype(np.float32)

# Hapus variabel Polars yang berat
del X_train, y_train, X_val, y_val
gc.collect() # Paksa buang sampah memori
print("   RAM dibersihkan. Lanjut Scaling.")

# 2. SCALING (In-Place)
print("2. Melakukan Scaling...")
scaler = StandardScaler()
# Fit & Transform langsung timpa variabel lama
X_train_np = scaler.fit_transform(X_train_np)
X_val_np = scaler.transform(X_val_np)
print("   Scaling selesai.")

# 3. CUSTOM DATASET (Kunci Hemat RAM)
# Kita tidak mengubah semua jadi Tensor sekaligus.
# Kita biarkan tetap Numpy, baru diubah jadi Tensor "sejumput-sejumput" pas mau masuk mesin.
class LazyFraudDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        # Nah, konversi ke Tensor baru terjadi di sini (per baris)
        x = torch.tensor(self.features[idx], dtype=torch.float32)
        y = torch.tensor(self.targets[idx], dtype=torch.float32).unsqueeze(0)
        return x, y

# Buat Dataset & Loader
batch_size = 2048 # Kita perbesar batch biar GPU kenyang
train_dataset = LazyFraudDataset(X_train_np, y_train_np)
val_dataset = LazyFraudDataset(X_val_np, y_val_np)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"   Data Loader Siap. Batch Size: {batch_size}")

# 4. ARSITEKTUR MODEL
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"   Device: {device}")

class SimpleFraudNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleFraudNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# Init Model
model_nn = SimpleFraudNN(X_train_np.shape[1]).to(device)
optimizer = optim.Adam(model_nn.parameters(), lr=0.001)
criterion = nn.BCELoss()

# 5. TRAINING LOOP
from sklearn.metrics import roc_auc_score

print("\n=== MULAI TRAINING (20 EPOCH) ===")
epochs = 20
best_auc = 0

for epoch in range(epochs):
    model_nn.train()
    total_loss = 0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model_nn(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Evaluasi Val
    model_nn.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.to(device)
            outputs = model_nn(inputs)

            y_true.extend(targets.cpu().numpy())
            y_pred.extend(outputs.cpu().numpy())

    val_auc = roc_auc_score(y_true, y_pred)

    # Simpan model terbaik
    if val_auc > best_auc:
        best_auc = val_auc
        torch.save(model_nn.state_dict(), "best_fraud_model.pth")

    print(f"Epoch {epoch+1:02d} | Loss: {total_loss/len(train_loader):.4f} | Val AUC: {val_auc:.4f}")

print("\n=== SELESAI BOS! ===")
print(f"Best Neural Network AUC: {best_auc:.4f}")

=== MULAI DEEP LEARNING MODE HEMAT RAM ===
1. Mengonversi ke Numpy (Hemat Memori)...
   RAM dibersihkan. Lanjut Scaling.
2. Melakukan Scaling...
   Scaling selesai.
   Data Loader Siap. Batch Size: 2048
   Device: cpu

=== MULAI TRAINING (20 EPOCH) ===
Epoch 01 | Loss: 0.1602 | Val AUC: 0.8347
Epoch 02 | Loss: 0.1051 | Val AUC: 0.8428
Epoch 03 | Loss: 0.1016 | Val AUC: 0.8563
Epoch 04 | Loss: 0.0989 | Val AUC: 0.8543
Epoch 05 | Loss: 0.0965 | Val AUC: 0.8548
Epoch 06 | Loss: 0.0950 | Val AUC: 0.8635
Epoch 07 | Loss: 0.0937 | Val AUC: 0.8556
Epoch 08 | Loss: 0.0927 | Val AUC: 0.8599
Epoch 09 | Loss: 0.0912 | Val AUC: 0.8685
Epoch 10 | Loss: 0.0899 | Val AUC: 0.8611
Epoch 11 | Loss: 0.0893 | Val AUC: 0.8671
Epoch 12 | Loss: 0.0881 | Val AUC: 0.8612
Epoch 13 | Loss: 0.0874 | Val AUC: 0.8600
Epoch 14 | Loss: 0.0863 | Val AUC: 0.8619
Epoch 15 | Loss: 0.0859 | Val AUC: 0.8659
Epoch 16 | Loss: 0.0849 | Val AUC: 0.8642
Epoch 17 | Loss: 0.0843 | Val AUC: 0.8661
Epoch 18 | Loss: 0.0837 | Val AUC

In [10]:
# --- CODE BLOCK: DEEP LEARNING (VERSI SUPER RINGAN / DEMO) ---
import polars as pl
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Path dataset (sesuaikan lagi kalau perlu)
BASE_PATH = "/content/drive/MyDrive/midterm_Tansaction"

print("=== DEEP LEARNING: DEMO MODE (50k Data Only) ===")
print("1. Loading Data (Hanya 50.000 baris agar RAM Aman)...")

# TRICK: Baca cuma 50k baris. Hasilnya gak maksimal, tapi CODE PASTI JALAN.
df = pl.read_csv(f"{BASE_PATH}/train_clean.csv", n_rows=50000)

# Persiapan Data
y = df["isFraud"].to_numpy().astype(np.float32)
X = df.drop(["isFraud", "TransactionID", "TransactionDT"]).to_numpy().astype(np.float32)

# Split Train/Val
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Convert to Tensor
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train).unsqueeze(1)
X_val_tensor = torch.tensor(X_val)
y_val_tensor = torch.tensor(y_val).unsqueeze(1)

# Dataset & Loader
train_ds = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True) # Batch kecil aja

# Model Arsitektur
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 1),
    nn.Sigmoid()
)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

# Training Loop Singkat (10 Epoch aja)
print("\n2. Mulai Training Singkat...")
for epoch in range(10):
    model.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        loss = criterion(model(xb), yb)
        loss.backward()
        optimizer.step()

    # Cek Akurasi kasar
    model.eval()
    with torch.no_grad():
        val_pred = model(X_val_tensor)
        val_loss = criterion(val_pred, y_val_tensor)

    print(f"Epoch {epoch+1}/10 | Val Loss: {val_loss.item():.4f}")

print("\n✅ SUKSES! Kode Deep Learning Berhasil Running.")
print("Catatan Laporan: Model dilatih pada subset data (50k) karena limitasi hardware.")

=== DEEP LEARNING: DEMO MODE (50k Data Only) ===
1. Loading Data (Hanya 50.000 baris agar RAM Aman)...

2. Mulai Training Singkat...
Epoch 1/10 | Val Loss: 0.0948
Epoch 2/10 | Val Loss: 0.0918
Epoch 3/10 | Val Loss: 0.0878
Epoch 4/10 | Val Loss: 0.0902
Epoch 5/10 | Val Loss: 0.0864
Epoch 6/10 | Val Loss: 0.0862
Epoch 7/10 | Val Loss: 0.0862
Epoch 8/10 | Val Loss: 0.0848
Epoch 9/10 | Val Loss: 0.0840
Epoch 10/10 | Val Loss: 0.0834

✅ SUKSES! Kode Deep Learning Berhasil Running.
Catatan Laporan: Model dilatih pada subset data (50k) karena limitasi hardware.


In [2]:
# --- CODE BLOCK FINAL: LOW MEMORY SUBMISSION STRATEGY ---
import polars as pl
import lightgbm as lgb
import pandas as pd
import gc
import numpy as np

print("=== FINAL STEP: GENERATE SUBMISSION (MODE HEMAT RAM) ===")

# --- TAHAP 1: TRAINING ---
print("\n1. [TRAINING FASE] Loading Data Train...")
# Trik Hemat: Ubah semua angka jadi Float32 (Ukuran data turun 50%)
train_df = pl.read_csv(f"{BASE_PATH}/train_clean.csv")
train_df = train_df.with_columns(pl.all().cast(pl.Float32))

print("   Data Train Loaded. Converting to LightGBM Dataset...")
X_train = train_df.drop(["isFraud", "TransactionID", "TransactionDT"]).to_pandas()
y_train = train_df["isFraud"].to_pandas()

# HAPUS POLARS DF SEGERA SETELAH JADI PANDAS
del train_df
gc.collect()

print("   Mulai Training LightGBM...")
dtrain = lgb.Dataset(X_train, label=y_train)

# Parameter (Sama kayak tadi)
params = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': True,
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 64,
    'verbose': -1
}

# Training
model = lgb.train(params, dtrain, num_boost_round=150) # 150 ronde cukup

# BERSIH-BERSIH RAM TOTAL SEBELUM PREDIKSI
print("   Training Selesai. Menghapus data training dari RAM...")
del X_train, y_train, dtrain
gc.collect()


# --- TAHAP 2: PREDICTION ---
print("\n2. [PREDICTION FASE] Loading Data Test...")
test_df = pl.read_csv(f"{BASE_PATH}/test_clean.csv")
test_df = test_df.with_columns(pl.all().cast(pl.Float32))

print("   Mengonversi Data Test...")
X_test = test_df.drop(["TransactionID", "TransactionDT"]).to_pandas()

# HAPUS POLARS TEST
del test_df
gc.collect()

print("   Melakukan Prediksi...")
preds = model.predict(X_test)

# HAPUS X_TEST (Sudah tidak butuh, kita cuma butuh hasil prediksinya)
del X_test
gc.collect()


# --- TAHAP 3: SAVING ---
print("\n3. [SAVING FASE] Mengambil TransactionID Asli...")
# Kita ambil ID-nya saja, pakai scan biar ringan
ids = pl.read_csv(f"{BASE_PATH}/test_transaction.csv", columns=["TransactionID"])["TransactionID"]

print("   Menyimpan submission_final.csv...")
submission = pd.DataFrame({
    'TransactionID': ids.to_numpy(),
    'isFraud': preds
})

save_path = f"{BASE_PATH}/submission_final.csv"
submission.to_csv(save_path, index=False)

print("\n=============================================")
print(f"✅ ALHAMDULILLAH SUKSES BOS!")
print(f"File siap dikumpul: {save_path}")
print("=============================================")
print(submission.head())

=== FINAL STEP: GENERATE SUBMISSION (MODE HEMAT RAM) ===

1. [TRAINING FASE] Loading Data Train...
   Data Train Loaded. Converting to LightGBM Dataset...
   Mulai Training LightGBM...
   Training Selesai. Menghapus data training dari RAM...

2. [PREDICTION FASE] Loading Data Test...
   Mengonversi Data Test...
   Melakukan Prediksi...

3. [SAVING FASE] Mengambil TransactionID Asli...
   Menyimpan submission_final.csv...

✅ ALHAMDULILLAH SUKSES BOS!
File siap dikumpul: /content/drive/MyDrive/midterm_Tansaction/submission_final.csv
   TransactionID   isFraud
0        3663549  0.053886
1        3663550  0.050637
2        3663551  0.034913
3        3663552  0.067506
4        3663553  0.063576
